In [2]:
import minigrid
from minigrid.wrappers import ImgObsWrapper
from stable_baselines3 import PPO
import torch.nn as nn

import torch
import gymnasium as gym
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from typing import Dict


pygame 2.5.1 (SDL 2.28.2, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [34]:
class MinigridFeaturesExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.Space, features_dim: int = 64, normalized_image: bool = False) -> None:
        super().__init__(observation_space, features_dim)
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 16, (2, 2)),
            nn.ReLU(),
            nn.Conv2d(16, 32, (2, 2)),
            nn.ReLU(),
            nn.Conv2d(32, 64, (2, 2)),
            nn.ReLU(),
            nn.Flatten(),
        )

        print(observation_space)
        
        n = observation_space.shape[0]
        m = observation_space.shape[1]
        print(n,m)
        self.image_embedding_size = ((n-1)//2-2)*((m-1)//2-2)*64


        # Compute shape by doing one forward pass
        tens= torch.as_tensor(observation_space.sample()[None]).float()
        print(tens.shape)
        with torch.no_grad():
            n_flatten = self.cnn(tens).shape[1]
        print(features_dim, self.image_embedding_size)
        lin = nn.Linear(n_flatten, features_dim)
        self.linear = nn.Sequential(lin, nn.ReLU())

    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        
        print(f"from_forward{observations.shape}")
        observations = torch.Tensor(observations)
        return self.linear(self.cnn(observations))

In [1]:
n = 7
m = 7
((n-1)//2-2)*((m-1)//2-2)*64

64

In [4]:
def get_obss_preprocessor(obs_space):
    # Check if obs_space is an image space
    if isinstance(obs_space, gym.spaces.Box):
        obs_space = {"image": obs_space.shape}

        def preprocess_obss(obss, device=None):
            return torch_ac.DictList({
                "image": preprocess_images(obss, device=device)
            })

    # Check if it is a MiniGrid observation space
    elif isinstance(obs_space, gym.spaces.Dict) and "image" in obs_space.spaces.keys():
        obs_space = {"image": obs_space.spaces["image"].shape, "text": 100}

        vocab = Vocabulary(obs_space["text"])

        def preprocess_obss(obss, device=None):
            return torch_ac.DictList({
                "image": preprocess_images([obs["image"] for obs in obss], device=device),
                "text": preprocess_texts([obs["mission"] for obs in obss], vocab, device=device)
            })

        preprocess_obss.vocab = vocab

    else:
        raise ValueError("Unknown observation space: " + str(obs_space))

    return obs_space, preprocess_obss


def preprocess_images(images, device=None):
    # Bug of Pytorch: very slow if not first converted to numpy array
    images = numpy.array(images)
    return torch.tensor(images, device=device, dtype=torch.float)

In [33]:


policy_kwargs = dict(
    features_extractor_class=MinigridFeaturesExtractor,
    features_extractor_kwargs=dict(features_dim=128),
)

env = gym.make("MiniGrid-Empty-16x16-v0")
env = ImgObsWrapper(env)



model = PPO("CnnPolicy", env, policy_kwargs=policy_kwargs, verbose=1)
model.learn(20)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Box(0, 255, (3, 7, 7), uint8)
3 7
torch.Size([1, 3, 7, 7])
128 -64
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Siz

In [39]:
obs = env.reset()
obs = torch.tensor(obs[0]).unsqueeze(0)
obs.shape

torch.Size([1, 7, 7, 3])

In [45]:
obs = env.reset()
obs = torch.tensor(obs[0]).unsqueeze(0)

feature_extractor  = MinigridFeaturesExtractor(env.observation_space, features_dim=64)
print(obs.shape)
obs =torch.tensor(obs).float().permute(0,3,1,2)

print(f"{obs.shape=}")
x = feature_extractor(obs)
obs, _ , _, _, _ = env.step(env.action_space.sample())
obs =torch.tensor(obs).float().unsqueeze(0).permute(0,3,1,2)
x = feature_extractor(obs)


Box(0, 255, (7, 7, 3), uint8)
7 7
torch.Size([1, 3, 7, 7])
64 64
torch.Size([1, 7, 7, 3])
obs.shape=torch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])
from_forwardtorch.Size([1, 3, 7, 7])


/var/folders/58/f16y6j7x0j5d0vv3pvq80c680000gn/T/ipykernel_46465/3424985998.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  obs =torch.tensor(obs).float().permute(0,3,1,2)


In [187]:
obs.transpose(0,2)

ValueError: axes don't match array

In [217]:
obs = env.reset()
obs = obs[0]
done = False
feature_extractor  = MinigridFeaturesExtractor(env.observation_space, features_dim=128)

while not done:
    action, _states = model.predict(obs)
    obs, rewards, done, info, _ = env.step(action)
    obs_copy = obs
    tensor_obs = torch.tensor(obs).float().permute(2,0,1).unsqueeze(0)
    features = feature_extractor(tensor_obs)
    print("done")


Box(0, 255, (7, 7, 3), uint8)
7 7
128 64


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1024 and 64x128)

In [183]:
env.observation_space

Box(0, 255, (7, 7, 3), uint8)

In [103]:
model.policy.features_extractor

MinigridFeaturesExtractor(
  (cnn): Sequential(
    (0): Conv2d(3, 16, kernel_size=(2, 2), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1))
    (5): ReLU()
    (6): Flatten(start_dim=1, end_dim=-1)
  )
  (linear): Sequential(
    (0): Linear(in_features=1024, out_features=128, bias=True)
    (1): ReLU()
  )
)

In [104]:
feature_extractor

NameError: name 'feature_extractor' is not defined

In [79]:
tensor_obs.shape

torch.Size([1, 3, 7, 7])

In [96]:
obs = env.reset()
obs[0].shape

(7, 7, 3)

In [90]:
obs[0].unsqueeze(0)

AttributeError: 'numpy.ndarray' object has no attribute 'unsqueeze'

In [100]:
obs_copy.shape

(7, 7, 3)